# 목차
### 1. 계층적 색인
### 2. 재형성과 피벗
### 3. Merge & Join 예제1
### 4. Merge & Join 예제2
### 5. 이어붙이기 (concat, concatenate)

## 계층적 색인
- 축에 대해 다중 색인 단계를 지정할 수 있게 해준다.

In [7]:
import pandas as pd
import numpy as np

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

data = pd.Series(np.random.randn(9),
                index = [['a','a','a','b','b','c','c','d','d'],
                         [ 1 , 2 , 3 , 1 , 3 , 1 , 2 , 2 , 3 ]])

data

a  1   -1.975643
   2    0.360560
   3   -0.594033
b  1    0.545741
   3   -0.094295
c  1   -0.006174
   2    0.290346
d  2    1.905070
   3   -0.470291
dtype: float64

In [12]:
## 데이터는 부분적 색인으로 접근 가능하다
print(data.index)

# data b 행
data['b']

# data b ~ d 행
data['b':'d']


MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )


1    0.545741
3   -0.094295
dtype: float64

b  1    0.545741
   3   -0.094295
c  1   -0.006174
   2    0.290346
d  2    1.905070
   3   -0.470291
dtype: float64

## 재형성과 피벗(pivot)
- 표 형식의 데이터를 재배치 하는 방법 (재형성 또는 피벗연산 이라고 함)

In [13]:
# data unstack
data_unstack = data.unstack()
data_unstack

# data stack
# dropna 버리기도 가능
data_stack = data_unstack.stack(dropna=False)
data_stack

,1,2,3
a,-1.975643,0.360560,-0.594033
b,0.545741,NaN,-0.094295
c,-0.006174,0.290346,NaN
d,NaN,1.905070,-0.470291


a  1   -1.975643
   2    0.360560
   3   -0.594033
b  1    0.545741
   3   -0.094295
c  1   -0.006174
   2    0.290346
d  2    1.905070
   3   -0.470291
dtype: float64

## 계층의 순서를 바꾸고 정렬하기
- 계층 바꾸기 swaplevel 함수
- 정렬하기 : sort_index -> 한 열에 대해서밖에 안됨.

In [17]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index = [['a','a','b','b'],[1, 2, 1, 2]],
                     columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])

# index, column name 설정
frame.index.names=['key1','key2']
frame.columns.names=['state','color']

frame

# 계층의 level 순서를 바꿈.
frame.swaplevel('key1','key2')

# 정렬하기
frame.sort_index(level=1)


state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

## 계층별 요약 통계

In [20]:
# 행 별 평균, key2 기준
frame.sum(level='key2')

# 열 별 평균 color 기준
frame.mean(level='color', axis=1)

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

color      Green  Red
key1 key2            
a    1         1    1
     2         4    4
b    1         7    7
     2        10   10

## 데이터 합치기 - merge 
- 공통된 컬럼이 있어야 한다. (이름은 안같더라도 data가 같은 형식인)
- 중복된 컬럼이름을 키로 사용한다.

- HOW 옵션에 따른 다양한 조인연산
    - inner : 양쪽 테이블 모두에 존재하는 키 조합 사용
    - left : 왼쪽 테이블에 존재하는 모든 키 조합 사용
    - right : 오른쪽 테이블에 존재하는 모든 키 조합 사용
    - outer: 양쪽 테이블에 존재하는 모든 키 사용
    
- **기본 merge는 inner 옵션일 때 공통된 행은 그냥 다 합쳐버린다. df1의 서로 column이 다른 항목이 각각 있더라도 key만 같으면 최종적으로 한 행렬로 합친다.** 

In [62]:
df1 = pd.DataFrame({'key':['b','b','a','c','a'],
                   'data1':range(5)})

df2 = pd.DataFrame({'key':['a','b','d'],
                   'data2':range(3),
                   'data3':[10,11,12]})

df1

df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4


,key,data2,data3
0,a,0,10
1,b,1,11
2,d,2,12


In [63]:
# left 옵션 -> key 값 기준으로 abcd 가 만들어짐
pd.merge(df1, df2, how='left')

# right 옵션 -> key 값 기준으로 abd만 만들어짐
pd.merge(df1, df2, how='right')

,key,data1,data2,data3
0,b,0,1.0,11.0
1,b,1,1.0,11.0
2,a,2,0.0,10.0
3,c,3,NaN,NaN
4,a,4,0.0,10.0


,key,data1,data2,data3
0,b,0.0,1,11
1,b,1.0,1,11
2,a,2.0,0,10
3,a,4.0,0,10
4,d,NaN,2,12


In [64]:
# inner 옵션 -> key 값 기준으로 공통된 것만 만들어짐 -> ab
pd.merge(df1, df2 ,how ='inner') # default = pd.merge(df1, df2, on='key')

# outer 옵션 -> key 값 기준 abcd 모두 사용됨
pd.merge(df1, df2, how='outer')

,key,data1,data2,data3
0,b,0,1,11
1,b,1,1,11
2,a,2,0,10
3,a,4,0,10


,key,data1,data2,data3
0,b,0.0,1.0,11.0
1,b,1.0,1.0,11.0
2,a,2.0,0.0,10.0
3,a,4.0,0.0,10.0
4,c,3.0,NaN,NaN
5,d,NaN,2.0,12.0


## 결합하려는 게 색인일 경우
- left_on : 조인키로 사용할 left DataFrame의 컬럼
- right_on : 조인키로 사용할 right DataFrame의 컬럼
- left_index : 조인키로 사용할 left DataFrame의 색인 로우
- right_index : 조인키로 사용할 right DataFrame의 색인 로우
- suffixes :  컬럼 이름 겹칠 경우 각 컬럼 이름 뒤에 붙일 문자열 튜플지정해주기
- 중복되는 색인값 사용시 outer 사용

In [51]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],
                      'value' : range(6)})

right1 = pd.DataFrame({'group_val':[3.5, 7]}, index=['a','b'])

left1

right1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


In [59]:
# left_on 왼쪽 dataFrame 의 key란 컬럼을 사용 / 
# right_index 오른쪽 DataFrame 의 index를 사용하겠다.
pd.merge(left1, right1, left_on='key',right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


## Join 메서드
- merge나 join 같은 역할
- **색인끼리 합칠 때 사용 = 색인이 공통 key 일 때 =========== merge(left_on = key, left_index=True) 형태와 유사**
- 컬럼이 겹치지 않으며, 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 떄 사용.

In [67]:
left2 = pd.DataFrame([[1,2,],[3,4],[5,6]],
                     index=['a','c','e'],
                     columns=['Ohio','Nevada'])

right2 = pd.DataFrame([[7, 8],[9,10],[11,12],[13,14]],
                      index=['b','c','d','e'],
                      columns = ['Missouri','Alabama'])

left2
right2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [69]:
# 공통 색인으로 겹칠떄 주로 사용
left2.join(right2, how='outer')

# 하나는 index 하나는 column
left1.join(right1, on='key')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


## merge & join 예제2 
- 출처 : https://rfriend.tistory.com/259
![join](__imgs/join.PNG)


In [71]:
import pandas as pd
df_left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3']},
                       index=['K0', 'K1', 'K2', 'K3'])

df_right = pd.DataFrame({'C': ['C2', 'C3', 'C4', 'C5'],
                         'D': ['D2', 'D3', 'D4', 'D5']},
                        index=['K2', 'K3', 'K4', 'K5'])
df_left
df_right

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2
K3,A3,B3


,C,D
K2,C2,D2
K3,C3,D3
K4,C4,D4
K5,C5,D5


In [72]:
# (1) index를 기준으로 Left Join 하기 (Left join on index)
# way 1 : merge()
pd.merge(df_left, df_right, left_index=True, right_index=True, how='left')

# way2 : join()
df_left.join(df_right, how='left')

,A,B,C,D
K0,A0,B0,NaN,NaN
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,A3,B3,C3,D3


,A,B,C,D
K0,A0,B0,NaN,NaN
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,A3,B3,C3,D3


In [73]:
# (2) index를 기준으로 Right Join 하기 (Right join on index)
# way 1 : merge()
pd.merge(df_left, df_right, left_index=True, right_index=True, how='right')

# way 2 : join()
df_left.join(df_right, how='right')


,A,B,C,D
K2,A2,B2,C2,D2
K3,A3,B3,C3,D3
K4,NaN,NaN,C4,D4
K5,NaN,NaN,C5,D5


,A,B,C,D
K2,A2,B2,C2,D2
K3,A3,B3,C3,D3
K4,NaN,NaN,C4,D4
K5,NaN,NaN,C5,D5


In [74]:
# (3) index를 기준으로 inner join 하기 (inner join on index)
# way 1 : merge()
pd.merge(df_left, df_right, left_index=True, right_index=True, how='inner')

# way 2 : join()
df_left.join(df_right, how='inner')

,A,B,C,D
K2,A2,B2,C2,D2
K3,A3,B3,C3,D3


,A,B,C,D
K2,A2,B2,C2,D2
K3,A3,B3,C3,D3


In [75]:
# (4) index를 기준으로 outer join 하기 (outer join on index)

# way 1 : merge()
pd.merge(df_left, df_right, left_index=True, right_index=True, how='outer')

# way 2 : join()
df_left.join(df_right, how='outer')

,A,B,C,D
K0,A0,B0,NaN,NaN
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,A3,B3,C3,D3
K4,NaN,NaN,C4,D4
K5,NaN,NaN,C5,D5


,A,B,C,D
K0,A0,B0,NaN,NaN
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,A3,B3,C3,D3
K4,NaN,NaN,C4,D4
K5,NaN,NaN,C5,D5


In [78]:
# (5) index와 Key를 혼합해서 DataFrame 합치기 (Joining key columns on an index)

df_left_2 = pd.DataFrame({'KEY': ['K0', 'K1', 'K2', 'K3'],
                                'A': ['A0', 'A1', 'A2', 'A3'],
                                'B': ['B0', 'B1', 'B2', 'B3']})
df_right_2 = pd.DataFrame({'C': ['C2', 'C3', 'C4', 'C5'],
                                 'D': ['D2', 'D3', 'D4', 'D5']},
                                index=['K2', 'K3', 'K4', 'K5'])
df_left_2
df_right_2 

# way 1 : merge()
pd.merge(df_left_2, df_right_2, left_on='KEY', right_index=True, how='left')

# way 2 : join()
df_left_2.join(df_right_2, on='KEY', how='left')

,KEY,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


,C,D
K2,C2,D2
K3,C3,D3
K4,C4,D4
K5,C5,D5


,KEY,A,B,C,D
0,K0,A0,B0,NaN,NaN
1,K1,A1,B1,NaN,NaN
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


,KEY,A,B,C,D
0,K0,A0,B0,NaN,NaN
1,K1,A1,B1,NaN,NaN
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


## 이어붙이기 (pd.concat), (np.concatenate)

In [80]:
arr = np.arange(12).reshape(3,4)
arr

# axis = 0  (아래쪽으로)
np.concatenate([arr,arr], axis=0)

# axis = 1  (옆쪽으로)
np.concatenate([arr,arr], axis=1)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [88]:
s1 = pd.Series([0,1], index=['a','b'])
s2 = pd.Series([2,3], index=['c','d'])
s3 = pd.Series([5,6], index=['a','b'])

# 아래로 이어붙이기
pd.concat([s1,s2,s3])

# 옆으로 이어붙이기 = index가 같을 떄만 가능
pd.concat([s1,s3], axis=1)

a    0
b    1
c    2
d    3
a    5
b    6
dtype: int64

,0,1
a,0,5
b,1,6
